In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time
import pandas as pd
from selenium.webdriver.support.ui import Select
from webdriver_manager.chrome import ChromeDriverManager

In [155]:
import time
import re

In [11]:
opciones=Options()
opciones.add_experimental_option('excludeSwitches', ['enable-automation'])
opciones.add_experimental_option('useAutomationExtension', False)
opciones.headless=False
opciones.add_argument('--start-maximized')
opciones.add_argument('--incognito')

In [14]:
cp = input("introduce tu código postal")

introduce tu código postal28009


In [183]:
driver = "./chromedriver.exe"
driver = webdriver.Chrome(driver,options=opciones)
url="https://www.rastreator.com/"
driver.get(url)
driver.implicitly_wait(3)

#aceptamos cookies en caso de que lo pidan:
try:
    driver.find_element_by_css_selector("#cookies > div > div.col-xs-12.col-sm-5.col-md-4.buttons > button.btn.solid-main.accept-btn").click()
except:
    pass

#seleccionamos que queremos comparar telefonía:
time.sleep(3)
driver.find_element_by_css_selector("body > div.overflow > section.products > div > div > div > div > div:nth-child(5)").click()

#seleccionamos que somos un particular:
time.sleep(3)
driver.find_element_by_css_selector("#qt-TI-14-at-27 > div > span").click()

#introducimos el código postal, esperamos, y hacemos click en "avanzar":
time.sleep(2)
driver.find_element_by_css_selector("#qt-TI-22-at-41").send_keys(cp)
time.sleep(3)
driver.find_element_by_css_selector("#next").click()

company = []
permanencia = []
megas = []
gigas = []
precio = []

time.sleep(5)
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
time.sleep(3)

for i in range(30):
    try: 
        company.append(driver.find_element_by_xpath(f"/html/body/app-root/app-quotes/app-quote-page/div[2]/div/div/div/div[3]/div/div[2]/module-quotes-detail/div[{i+1}]/module-quote/div/div/div[2]/div/div/a/img").get_attribute("src").split('/')[-1].split('.')[0])
        permanencia.append(driver.find_element_by_css_selector(f"#Quote_{i} > div > div.price-block-container > div > div > div.tel-condition > div:nth-child(1) > p").text)
        megas.append(driver.find_element_by_css_selector(f"#Quote_{i} > div > div.price-block-container > div > div > div.tel-condition > div.tel-feature > ul > li:nth-child(1) > span").text)
        gigas.append(driver.find_element_by_css_selector(f"#Quote_{i} > div > div.price-block-container > div > div > div.tel-condition > div.tel-feature > ul > li:nth-child(3) > span").text)
        precio.append(driver.find_element_by_css_selector(f"#Quote_{i} > div > div.price-block-container > div > div > div.price-wrapper > div > p > span.amount").text)
    except:
        pass
        
#driver.quit()

<ipython-input-183-edbe29341d4c>:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(driver,options=opciones)
<ipython-input-183-edbe29341d4c>:9: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_css_selector("#cookies > div > div.col-xs-12.col-sm-5.col-md-4.buttons > button.btn.solid-main.accept-btn").click()
<ipython-input-183-edbe29341d4c>:15: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_css_selector("body > div.overflow > section.products > div > div > div > div > div:nth-child(5)").click()
<ipython-input-183-edbe29341d4c>:19: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_css_selector("#qt-TI-14-at-27 > div > span").click()
<ipython-input-183-edbe29341d4c>:23: DeprecationWarning: find_element_b

In [184]:
company=["Lowi" if com=="Nuevologo_Lowi_68x48" else com for com in company]
permanencia=[p.split(" ")[0]+" "+p.split(" ")[1] for p in permanencia]
megas=[p.split(" ")[0] for p in megas]
gigas=['+'.join(re.findall(r"\d*GB", elem)) for elem in gigas]

In [186]:
ofertas = pd.DataFrame()
ofertas = ofertas.append(pd.DataFrame(company))
ofertas = pd.concat([ofertas, pd.DataFrame(permanencia)], axis=1)
ofertas = pd.concat([ofertas, pd.DataFrame(gigas)], axis=1)
ofertas = pd.concat([ofertas, pd.DataFrame(megas)], axis=1)
ofertas = pd.concat([ofertas, pd.DataFrame(precio)], axis=1)
ofertas.columns = ["Compañía", "permanencia", "Gigas_móvil", "Megas_fibra", "€/mes"]

In [189]:
ofertas.sample(3)

,Compañía,permanencia,Gigas_móvil,Megas_fibra,€/mes
3,o2,Sin permanencia.,10GB,100Mb,"30,00"
23,masmovil,3 meses,35GB,100Mb,"39,90"
10,o2,Sin permanencia.,25GB,300Mb,"38,00"


In [188]:
df.to_csv("DATA/ofertas.csv")